In [1]:
# train svm model

In [2]:
import sys
import pandas as pd
sys.path.append("../src")


from create_folds import create_folds
from train import run,process

import config

In [3]:
train_df = pd.read_csv("../data/fraudTrain.csv")

In [4]:
train_df = create_folds(train_df, True)
train_df = process(train_df)

../src\utilities\feature_engineering.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["week"] = df[date_col].dt.week


__COL_TO_DROP__: ['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'zip', 'trans_date_trans_time', 'dob', 'trans_num', 'unix_time', 'job', 'merch_long', 'merch_lat']


In [5]:

# train the model using the goldout strategy for validation
model = "random_forest_classifier"
error=["f1_score","auc"]
clfs = [run(train_df, fold, model, error) for fold in train_df.kfold.unique()]

********** Train for fold number:  0 **********
Shape of the training data: 648337, Shape of testing data: 648338
Error/metric: f1_score Fold=0, train_error=1.0, valid_error=0.804
Error/metric: auc Fold=0, train_error=1.0, valid_error=0.84
********** Train for fold number:  1 **********
Shape of the training data: 648338, Shape of testing data: 648337
Error/metric: f1_score Fold=1, train_error=1.0, valid_error=0.812
Error/metric: auc Fold=1, train_error=1.0, valid_error=0.848
